In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import gc
import re
import time
import warnings
warnings.filterwarnings("ignore")

In [1]:
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [2]:
tic = time.time()
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']

invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']

question_info = pd.read_table(os.path.join(DATA_PATH, 'question_info_0926.txt'), header=None)
question_info.columns = ['问题ID',  '问题创建时间' , '问题标题的单字编码序列' , '问题标题的切词编码序列' , '问题描述的单字编码序列',  '问题描述的词编码序列' , '问题绑定的话题ID']

answer_info = pd.read_table(os.path.join(DATA_PATH, 'answer_info_0926.txt'), header=None)
answer_info.columns = ['回答ID','问题ID','用户ID', '回答创建时间' ,'回答内容的单字编码序列', '回答内容的切词编码序列' ,'回答是否被标优', '回答是否被推荐' ,'回答是否被收入圆桌', '是否包含图片' ,'是否包含视频', '回答字数' ,'点赞数', '取赞数' ,'评论数' ,'收藏数', '感谢数' ,'举报数', '没有帮助数' ,'反对数']

oversample = False

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)
    
data = data.merge(question_info, on='问题ID', how='left')

print("Used time: %d s" % (time.time()-tic))

Used time: 110 s


In [3]:
tic = time.time()
tmp = data['邀请创建时间'].apply(lambda x : x.split('-'))
data['邀请创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
data['邀请创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))

tmp = answer_info['回答创建时间'].apply(lambda x : x.split('-'))
answer_info['回答创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
answer_info['回答创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))

data['id'] = np.arange(len(data))
answer_info['atime'] = answer_info['回答创建时间_D'] * 24 + answer_info['回答创建时间_H']
data['itime'] = data['邀请创建时间_D'] * 24 + data['邀请创建时间_H']
inv = data[['id', '用户ID', 'itime']]
ans = answer_info[['回答ID','问题ID','用户ID','回答创建时间_D','回答创建时间_H','atime','回答内容的单字编码序列', '回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐' ,'回答是否被收入圆桌', '是否包含图片' ,'是否包含视频', '回答字数' ,'点赞数', '取赞数' ,'评论数' ,'收藏数', '感谢数' ,'举报数', '没有帮助数' ,'反对数']]
inv.sort_values(by=['用户ID', 'itime'], inplace=True)
ans.sort_values(by=['用户ID', 'atime'], inplace=True)

inv.columns = ['id', '用户ID','time']
ans.columns = ['回答ID','问题ID','用户ID','回答创建时间_D','回答创建时间_H', 'time','回答内容的单字编码序列', '回答内容的切词编码序列','回答是否被标优', '回答是否被推荐' ,'回答是否被收入圆桌', '是否包含图片' ,'是否包含视频', '回答字数' ,'点赞数', '取赞数' ,'评论数' ,'收藏数', '感谢数' ,'举报数', '没有帮助数' ,'反对数']

tmp = pd.concat([inv, ans])
tmp.sort_values(by=['用户ID', 'time'], inplace=True)

print("Used time: %d s" % (time.time()-tic))

Used time: 102 s


In [4]:
def last_ans_stats(tmp):
    last_author = '-1'
    ts = []
    for id ,author, ti, answer_words in tqdm(tmp.values):
        if author != last_author:
            answer_list = []
            if pd.isna(id):
                answer_list.append(int(answer_words))
            ts.append(np.nan)

        elif pd.isna(id):
            if len(answer_list)!=0:
                ts.append(list(answer_list))
            else:
                ts.append(np.nan)
            answer_list.append(int(answer_words))

        else:
            if len(answer_list)!=0:
                ts.append(list(answer_list))
            else:
                ts.append(np.nan)

        last_author = author

    tmp['answer_words'] = ts
    return tmp['answer_words']

In [ ]:
tic = time.time()
tmpp = tmp[['id','用户ID','time','time']]
res = last_ans_stats(tmpp)
tmp['times'] = res

tmpp = tmp[['id','用户ID','time','回答创建时间_H']]
res = last_ans_stats(tmpp)
tmp['Hs'] = res

tmp = tmp[~tmp['id'].isnull()]
tmp.sort_values(by='id', inplace=True)

print("Used time: %d s" % (time.time()-tic))


Used time: 178 s


In [ ]:
tic = time.time()
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize()

tmp['times_mean'] = tmp['times'].parallel_apply(lambda x: np.mean(x))
tmp['times_max'] = tmp['times'].parallel_apply(lambda x: np.max(x))
tmp['times_min'] = tmp['times'].parallel_apply(lambda x: np.min(x))
tmp['times_std'] = tmp['times'].parallel_apply(lambda x: np.std(x))

tmp['Hs_mean'] = tmp['Hs'].parallel_apply(lambda x: np.mean(x))
tmp['Hs_max'] = tmp['Hs'].parallel_apply(lambda x: np.max(x))
tmp['Hs_min'] = tmp['Hs'].parallel_apply(lambda x: np.min(x))
tmp['Hs_std'] = tmp['Hs'].parallel_apply(lambda x: np.std(x))

tmp = tmp[['times_mean','times_max','times_min','times_std','Hs_mean','Hs_max','Hs_min','Hs_std']]
print("Used time: %d s" % (time.time()-tic))

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
tmp.to_pickle(os.path.join(DATA_PATH, 'pres_ans_time_stats.pkl'))
print("Feature Saved, shape:",tmp.shape)